In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
from resnet18 import ResNet, BasicBlock
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import cv2
import pickle

%matplotlib inline

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_resnet18_model(model_path, num_classes=2, num_channels=3):
    """

    :param model_path: Path to ResNet18 model
    :param num_channels: Number of channels in image, also depends on trained model
    :return: Instance of the model (torchvision.models.resnet18() in eval mode)
    """

    # Create an instance of the ResNet18 model
    model = ResNet(img_channels=num_channels, num_layers=18, block=BasicBlock, num_classes=num_classes).to(device)

    # Load the saved state dictionary into the model
    # map_location = device makes code compatible for CPU and GPU
    model.load_state_dict(torch.load(model_path, map_location=device))

    # Set the model to evaluation mode
    model.eval()

    return model


def load_image_with_transform(input_img, num_channels=3):
    input_img = input_img.copy()

    # input_img.resize((224, 224, 3))
    # input_img = input_img * 255 / np.max(input_img)
    # input_img = input_img.astype(np.uint8)

    input_img = Image.fromarray(input_img)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
        # transforms.Grayscale(num_output_channels=1)
    ])

    input_img = transform(input_img)

    n = 224

    input_img = torch.tensor(input_img.clone().detach().requires_grad_(True), dtype=torch.float)
    input_img = input_img.view(1, num_channels, n, n).to(device)

    return input_img

In [3]:
with open('/home/raja/Desktop/MICCAI/code/18_resnet/models/best.pkl', 'rb') as f:
    model = pickle.load(f)


In [4]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [5]:
'''
img = plt.imread('data/NONLYMPHOCYTES/img_2_cropped_30.jpg')
transformed_img =  load_image_with_transform(img, num_channels=3)

outputs = model(transformed_img)
outputs = F.softmax(outputs, dim=1)
predicted_class = torch.argmax(outputs, dim=1)
confidence_scores = torch.nn.functional.softmax(outputs, dim=1)

print(outputs)

_, preds = torch.max(outputs.data, 1)

#classes = ['Lymphocyte', 'Non-Lymphocyte']
classes = [0,1]

class_prediction = classes[preds.tolist()[0]]

class_prediction
'''

"\nimg = plt.imread('data/NONLYMPHOCYTES/img_2_cropped_30.jpg')\ntransformed_img =  load_image_with_transform(img, num_channels=3)\n\noutputs = model(transformed_img)\noutputs = F.softmax(outputs, dim=1)\npredicted_class = torch.argmax(outputs, dim=1)\nconfidence_scores = torch.nn.functional.softmax(outputs, dim=1)\n\nprint(outputs)\n\n_, preds = torch.max(outputs.data, 1)\n\n#classes = ['Lymphocyte', 'Non-Lymphocyte']\nclasses = [0,1]\n\nclass_prediction = classes[preds.tolist()[0]]\n\nclass_prediction\n"

In [6]:
class_predictions = []

for img_name in os.listdir('/home/raja/Desktop/MICCAI/test_folder'):
    img = plt.imread(os.path.join('/home/raja/Desktop/MICCAI', 'test_folder', img_name))
    transformed_img = load_image_with_transform(img, num_channels=3)

    outputs = model(transformed_img)
    outputs = F.softmax(outputs, dim=1)

    _, preds = torch.max(outputs.data, 1)

    classes = [0, 1]
    class_prediction = classes[preds.tolist()[0]]

    class_predictions.append(class_prediction)

#print(class_predictions)


/tmp/ipykernel_105286/499375677.py:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_img = torch.tensor(input_img.clone().detach().requires_grad_(True), dtype=torch.float)


In [7]:
import csv
import os

# Combine image names and predicted labels
data = zip(os.listdir('/home/raja/Desktop/MICCAI/test_folder'), class_predictions)

# Write to CSV file
with open('/home/raja/Desktop/MICCAI/code/18_resnet/predictions-resnet.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Image Name', 'Label'])
    for row in data:
        writer.writerow(row)


In [8]:
import csv
from collections import defaultdict
import re
# Initialize a dictionary to store the counts
counts = defaultdict(int)

# Read the CSV file and populate the counts dictionary
with open('/home/raja/Desktop/MICCAI/code/18_resnet/predictions-resnet.csv', 'r') as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row if present

    # Iterate over the rows in the CSV
    for row in reader:
        image_name = row[0]
        label = int(row[1])

        # Extract the image number
        match = re.search(r'img_(\d+)', image_name)
        if match:
            image_number = int(match.group(1))

            # Check if the label is 0
            if label == 0:
                counts[image_number] += 1

# Add missing image numbers with count 0
for image_number in range(18000, 20000):
    if image_number not in counts:
        counts[image_number] = 0

# Write the counts to a new CSV file
with open('/home/raja/Desktop/MICCAI/code/18_resnet/image_counts.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['img_number', 'count'])  # Write header row

    # Write the image numbers and counts
    for image_number, count in sorted(counts.items()):
        writer.writerow([image_number, count])

print("CSV file 'image_counts.csv' has been created.")


CSV file 'image_counts.csv' has been created.


In [9]:

'''
import pandas as pd

# read the csv file into a pandas DataFrame
df = pd.read_csv('/home/raja/Desktop/MICCAI/code/18_resnet/output-resnet.csv')

# create a new DataFrame with the full range of image numbers
img_range = range(20000)
new_df = pd.DataFrame({'Image': ['img_' + str(i) for i in img_range], 'Predicted-Lymphocyte': 0})

# merge the new DataFrame with the original DataFrame using a left join
merged_df = pd.merge(new_df, df, on='Image', how='left')


# write the merged DataFrame to a new csv file
merged_df.to_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file-itr2.csv', index=False)

# Load the Excel file
df = pd.read_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file-itr2.csv')

# Fill the 'predicted' column with 0 for NA values
df['Predicted-Lymphocyte_y'].fillna(0, inplace=True)

# Save the modified DataFrame back to Excel
df.to_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file-itr2.csv', index=False)
'''

"\nimport pandas as pd\n\n# read the csv file into a pandas DataFrame\ndf = pd.read_csv('/home/raja/Desktop/MICCAI/code/18_resnet/output-resnet.csv')\n\n# create a new DataFrame with the full range of image numbers\nimg_range = range(20000)\nnew_df = pd.DataFrame({'Image': ['img_' + str(i) for i in img_range], 'Predicted-Lymphocyte': 0})\n\n# merge the new DataFrame with the original DataFrame using a left join\nmerged_df = pd.merge(new_df, df, on='Image', how='left')\n\n\n# write the merged DataFrame to a new csv file\nmerged_df.to_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file-itr2.csv', index=False)\n\n# Load the Excel file\ndf = pd.read_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file-itr2.csv')\n\n# Fill the 'predicted' column with 0 for NA values\ndf['Predicted-Lymphocyte_y'].fillna(0, inplace=True)\n\n# Save the modified DataFrame back to Excel\ndf.to_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file-itr2.csv', index=False)\n"

In [10]:
'''
import pandas as pd

# read the input CSV file into a pandas dataframe
df = pd.read_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file.csv')

# define the function to assign class labels based on ground truth values
def assign_class(gt):
    if gt == 0:
        return '0 (None)'
    elif 1 <= gt <= 5:
        return '1-5'
    elif 6 <= gt <= 10:
        return '6-10'
    elif 11 <= gt <= 20:
        return '11-20'
    elif 21 <= gt <= 50:
        return '21-50'
    elif 51 <= gt <= 200:
        return '51-200'
    else:
        return '>200'

# assign class labels to ground truth values
df['Ground-truth Class'] = df['Ground-truth'].apply(assign_class)

# assign class labels to predicted values based on the constraints
df['Predicted Class'] = df.apply(lambda row: assign_class(row['Predicted-Lymphocyte_y']), axis=1)

# calculate the confusion matrix
confusion_matrix = pd.crosstab(df['Ground-truth Class'], df['Predicted Class'])

# define the desired order of the classes
desired_order = ['0 (None)', '1-5', '6-10', '11-20', '21-50']

# reindex the confusion matrix to match the desired order
confusion_matrix = confusion_matrix.reindex(index=desired_order, columns=desired_order, fill_value=0)

# print the confusion matrix
print(confusion_matrix)

'''

"\nimport pandas as pd\n\n# read the input CSV file into a pandas dataframe\ndf = pd.read_csv('/home/raja/Desktop/MICCAI/code/18_resnet/new_file.csv')\n\n# define the function to assign class labels based on ground truth values\ndef assign_class(gt):\n    if gt == 0:\n        return '0 (None)'\n    elif 1 <= gt <= 5:\n        return '1-5'\n    elif 6 <= gt <= 10:\n        return '6-10'\n    elif 11 <= gt <= 20:\n        return '11-20'\n    elif 21 <= gt <= 50:\n        return '21-50'\n    elif 51 <= gt <= 200:\n        return '51-200'\n    else:\n        return '>200'\n\n# assign class labels to ground truth values\ndf['Ground-truth Class'] = df['Ground-truth'].apply(assign_class)\n\n# assign class labels to predicted values based on the constraints\ndf['Predicted Class'] = df.apply(lambda row: assign_class(row['Predicted-Lymphocyte_y']), axis=1)\n\n# calculate the confusion matrix\nconfusion_matrix = pd.crosstab(df['Ground-truth Class'], df['Predicted Class'])\n\n# define the desired 